<a href="https://colab.research.google.com/github/jmyth742/EEG-BCI/blob/master/WorkingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydrive

In [0]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import time



##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus

In [3]:

## END OF IMPORTS
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#file_id = '1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896'
file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
#https://drive.google.com/open?id=1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
#https://drive.google.com/open?id=1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')
#mat = loadmat('train3.mat')
mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)
data=[]

o=mat['o']

data=mat['o'].data
data_set=mat['o'].data
#print(data[200][21])
labels=mat['o'].marker

newlabels = np.zeros([data.shape[0],4])

pos=np.zeros((22,667000), dtype=int)
found = np.zeros([1,])
#clean the shit out of it
while np.amax(labels) > 3:
    ind = np.argmax(labels)
    found = np.append(found,np.array([ind]), axis=0)
    labels[ind] = 0
    data[ind,:] = np.zeros([22,])
    
print(data.shape)   
#print(np.amax(labels))
#print(found.shape)
#print(found)

inside = 0
for y in labels:
  if inside % 10000 == 0:
    #print(inside)
    newlabels[inside,y] = 1
    inside += 1

labeled_data = (data,labels)

clean_data= np.hstack((data,labels.reshape((labels.shape[0],1))))
data= np.hstack((data,labels.reshape((labels.shape[0],1))))

clean_data = clean_data.astype('float32')
data = data.astype('float32')
print(data.shape)

(667000, 22)
(667000, 23)


In [0]:
def sequence_builder(seq_len, batch, data):
  seq = torch.zeros(seq_len,batch,23)
  j=0
  idx=0
  #seq = torch.zeros(time_seq,batch_size//time_seq,22)  
  seq = np.zeros([seq_len,batch,23])
  #for i in range(trim_start,finish_trim):
  for i in range(len(data),1):
    #if data[i][22] !=0:
      #start = i -100
    seq[idx][j][:] = data[i][:]
    idx=idx+1
    if idx == time_seq-1:
      idx=0
      j=j+1       

  return seq
  
  
  
#def label_seq_builder():
  

In [0]:
class eeg_LSTM(nn.Module):
    def __init__(self):
        super(eeg_LSTM, self).__init__()
        self.lstm = nn.LSTM(22, 4, 2)
        #torch.nn.init.xavier_normal_(self.lstm.weight)
        hidden = (torch.randn(2, 1, 4),
                  torch.randn(1, 1, 4))
    def forward(self, x):
        #for i in inputs:
        r_out, _ = self.lstm(i, hidden)
        output, hidden = F.softmax(r_out) 
          
        return output
      
      
class SimpleLSTM(nn.Module):
    def __init__(self, input_size=22, hidden_size=22, output_size=4,
                 device='cuda:0'):
        super(SimpleLSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.LSTM(input_size, 22,hidden_size,bias=False,batch_first=False)
        #self.i2o = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden=None, cell=None, device='cuda:0'):
        outputs = []
        if hidden==None:
          #print("hidden input hidden:",input.shape[1])
          hidden = self.init_hidden(input.shape[1]) 
          hidden.to(device)
        if cell==None:
          #print("hidden input cell:",input.shape[1])
          cell = self.init_hidden(input.shape[1])
          #cell.to(device)
        #for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
        output, (_,_)= self.i2h(input, (hidden,cell))

        #output = self.i2o(output)
          #print('output',output.shape)
        timesteps, batch, C = output.size()
        out = output.view(timesteps*batch,C)
          #outputs += [out]
          #print('shape output', output.shape)
        #output = F.softmax(output, dim=0)

        outputs = output.view(1,timesteps*batch,C)
        return outputs

    def init_hidden(self,shape=1, device='cuda:0'):
        return torch.zeros(22, shape, self.hidden_size).to(device)
      
    def init_cell(self,shape=1, device='cuda:0'):
        return torch.zeros(22, shape, self.hidden_size).to(device)
      


In [0]:
def get_optimizer(net, lr):
  optimizer = torch.optim.Adam(net.parameters(), lr=0.00001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.00001, amsgrad=False)
  #optimizer = torch.optim.SGD(net.parameters(), lr=0.0000000001, weight_decay=0.9, momentum=0.9)
  return optimizer

def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [0]:
def get_data(batch_size,test_batch_size=256,step=1):
  full_data = data
   
  #print(full_data.shape)
  # Create train and validation splits
  num_samples = len(full_data)
  training_samples = int(num_samples)
  #validation_samples = num_samples - training_samples  
  

  training=data[:training_samples]
  #print(training.shape)
  #validation=data[training_samples:]

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training, batch_size, shuffle=False)
  #val_loader = torch.utils.data.DataLoader(validation, test_batch_size, shuffle=False)
  #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
  
  return train_loader#, val_loader

In [0]:
import time 
def test(time_seq, batch_size, net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  print("test func")
  features = 22
  i = 0
  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  #xtrain  = torch.zeros(time_seq,batch_size//time_seq,features
  with torch.no_grad():
    for batch_idx, (inputs) in enumerate(data_loader):
      
      xtrain = inputs[:,:22]# = inputs[:,0:22]
      labels = inputs[:,22:23]
      """if batch_idx % 50 == 0:
        
        print('xtrain 0 ', xtrain[0])
        print('labels',labels[0])"""
        
        
      train = Variable(torch.FloatTensor(xtrain.reshape(290,5,22))).to(device)#.reshape(batch size, seq len, channels)

      output_seq = net(train)

      
      labels = labels.reshape(1450)

      targets = Variable(torch.LongTensor(labels.long())).to(device)
      loss = cost_function(output_seq[-1], targets)
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = output_seq[-1].max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(time_seq, batch_size,net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  print("train function ----")
  features = 22
  xtrain  = torch.zeros(time_seq,batch_size,features)
  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  #for i in range((step//batch_size)-1):

  for batch_idx, (inputs) in enumerate(data_loader):
    # Reset the optimizer
      optimizer.zero_grad()
    
      xtrain = inputs[:,:22]# = inputs[:,0:22]
      labels = inputs[:,22:23]
      """if batch_idx % 50 == 0:
        
        print('xtrain 0 ', xtrain[0])
        print('labels',labels[0])"""
      
      train = Variable(torch.FloatTensor(xtrain.reshape(290,5,22))).to(device)#.reshape(batch size, seq len, channels)

      output_seq = net(train)
      
      labels = labels.reshape(1450)

      targets = Variable(torch.LongTensor(labels.long())).to(device)
      loss = cost_function(output_seq[-1], targets)
      
      # Backward pass
      loss.backward()
    
      # Update parameters
      optimizer.step()
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = output_seq[-1].max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


In [0]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=1450, 
         device='cuda:0', 
         learning_rate=0.001, 
         time_seq=29,
         wd=0.9,
         epochs=200):
  
  #train_loader, val_loader = get_data(batch_size,batch_size)
  train_loader = get_data(batch_size,batch_size)
  net = SimpleLSTM().to(device)
  #net = SimpleLSTM()#.to(device)
  e=0
  optimizer = get_optimizer(net, learning_rate)
  
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(time_seq, batch_size, net, train_loader, cost_function)
  #val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.10f}, Training accuracy {:.10f}'.format(train_loss, train_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  for e in range(epochs):
    train_loss, train_accuracy = train(time_seq, batch_size,net, train_loader, optimizer, cost_function)
    #val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}',e)
    print('\t Training loss {:.10f}, Training accuracy {:.10f}'.format(train_loss, train_accuracy))
    #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  #train_loss, train_accuracy = test(net, train_loader, cost_function)
  #val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')


In [0]:
main()

Before training:
test func
	 Training loss 0.0021317533, Training accuracy 10.3442278861
-----------------------------------------------------
train function ----
Epoch: {:d} 0
	 Training loss 0.0021317533, Training accuracy 10.5124437781
-----------------------------------------------------
train function ----
Epoch: {:d} 1
	 Training loss 0.0021317533, Training accuracy 10.7890554723
-----------------------------------------------------
train function ----
Epoch: {:d} 2
	 Training loss 0.0021317533, Training accuracy 11.0106446777
-----------------------------------------------------
train function ----
Epoch: {:d} 3
	 Training loss 0.0021317533, Training accuracy 11.1929535232
-----------------------------------------------------
train function ----
Epoch: {:d} 4
	 Training loss 0.0021317533, Training accuracy 11.3812593703
-----------------------------------------------------
train function ----


In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


if __name__ == '__main__':
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    data1 = data.astype('double')
    data1 = data1[:100,:22]
    input = torch.from_numpy(data[3:, :-1])
    target = torch.from_numpy(data[3:, 1:])
    test_input = torch.from_numpy(data[:3, :-1])
    test_target = torch.from_numpy(data[:3, 1:])
    # build the model
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
    #begin to train
    for i in range(15):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            print('target shape ', target.shape)
            print('out shape ', out.shape)
            loss = criterion(out, target)
            print('loss:', loss.item())
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict, no need to track gradient here
        with torch.no_grad():
            future = 1000
            print(test_input.shape)
            pred = seq(test_input, future=future)
            loss = criterion(pred[:, :-future], test_target)
            print('test loss:', loss.item())
            y = pred.detach().numpy()
        # draw the result
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        plt.savefig('predict%d.pdf'%i)
        plt.close()

STEP:  0


RuntimeError: ignored

In [0]:
import time 
def test(time_seq, batch_size, net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  print("test func")
  features = 22
  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  #xtrain  = torch.zeros(time_seq,batch_size//time_seq,features
  with torch.no_grad():
    for batch_idx, (inputs) in enumerate(data_loader):

      ## lets turn the data into batches for the LSTM
      formatted = sequence_builder(145,23,inputs)
      xtrain = formatted[:,:,:22]# = inputs[:,0:22]
      labels = formatted[:,:,22:23]
      lab = torch.zeros(23,1)
      for j in range(23):
        #print(labels[:][j][0].max())
        lab[j][0] = labels[:][j][0].max()
      train = Variable(torch.FloatTensor(xtrain)).to(device)#.reshape(batch size, seq len, channels)

      
      output_seq = net(train)
      
      targets = Variable(torch.LongTensor(lab.reshape([23]).long())).to(device)
      
      loss = cost_function(output_seq[-1], targets)
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = output_seq[-1].max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(time_seq, batch_size,net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  print("train func")
  features = 22
  xtrain  = torch.zeros(time_seq,batch_size,features)
  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  #for i in range((step//batch_size)-1):
  for batch_idx, (inputs) in enumerate(data_loader):
     # Load data into GPU
      ## lets turn the data into batches for the LSTM
      formatted = sequence_builder(145,23,inputs)
      xtrain = formatted[:,:,:22]# = inputs[:,0:22]
      labels = formatted[:,:,22:23]
      lab = torch.zeros(23,1)
      for j in range(23):
        #print(labels[:][j][0].max())
        lab[j][0] = labels[:][j][0].max()
      train = Variable(torch.FloatTensor(xtrain)).to(device)#.reshape(batch size, seq len, channels)

      output_seq = net(train)
      
      targets = Variable(torch.LongTensor(lab.reshape([23]).long())).to(device)

      
      loss = cost_function(output_seq[-1], targets)
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = output_seq[-1].max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


In [81]:
inputs = [autograd.Variable(torch.randn((1, 3)))
          for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 1, 3))))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropogate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(
    torch.randn((1, 1, 3))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

NameError: ignored

In [0]:
time_seq=400
batch_size = 1586
trim_start = 35700
finish_trim = 634400
i= 35700
idx=0
j=0
#seq = torch.zeros(time_seq,batch_size//time_seq,22)  
seq = np.zeros([time_seq,batch_size,23])
for i in range(trim_start,finish_trim):
  if clean_data[i][22] !=0:
    start = i -100
    seq[idx][j][:] = clean_data[start][:]
    idx=idx+1
    if idx == 399:
      idx=0
      j=j+1
print(seq.shape)    